In [108]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix

In [87]:
sf = pd.read_csv('ca_san_francisco_2020_04_01.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
sf.shape

(905070, 24)

In [102]:
sf.dropna(how='any')

,raw_row_number,date,time,location,lat,lng,district,subject_age,subject_race,subject_sex,...,citation_issued,warning_issued,outcome,contraband_found,search_conducted,search_vehicle,search_basis,reason_for_stop,raw_search_vehicle_description,raw_result_of_contact_description
33182,50,2007-01-01,13:10:00,GENEVA/MISSION,37.716407,-122.441087,H,39.0,hispanic,male,...,False,False,arrest,False,True,True,other,Moving Violation,"Vehicle Inventory, Negative Result",In Custody Arrest
33185,53,2007-01-01,13:57:00,13TH ST / MISSION ST,37.769954,-122.419962,D,24.0,hispanic,female,...,False,False,arrest,True,True,True,other,Mechanical or Non-Moving Violation (V.C.),"Search Incident to Arrest, Positive Result",In Custody Arrest
33192,59,2007-01-01,14:45:00,GREAT HWY/FULTON,37.771329,-122.511123,G,24.0,white,male,...,False,True,warning,False,True,True,other,Mechanical or Non-Moving Violation (V.C.),Searched as a result of Probation or Parole Co...,Warning
33210,76,2007-01-01,19:40:00,LEAVENWORTH AND GOLDEN GATE,37.781878,-122.413880,J,27.0,white,male,...,False,True,warning,False,True,True,consent,Mechanical or Non-Moving Violation (V.C.),"Search with Consent, Negative Result",Warning
33211,77,2007-01-01,20:00:00,EVANS AT THIRD STREET,37.742817,-122.388026,C,48.0,black,male,...,True,False,citation,False,True,True,other,Mechanical or Non-Moving Violation (V.C.),"Vehicle Inventory, Negative Result",Citation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885924,768645,2016-06-30,20:00:00,442 GEARY,37.787168,-122.410733,A,59.0,hispanic,male,...,True,False,citation,True,True,True,other,Moving Violation,"Search without Consent, Positive Result",Citation
885927,768648,2016-06-30,20:05:00,HYDE AT GOLDENGATE,37.781668,-122.415524,J,35.0,asian/pacific islander,male,...,False,True,warning,False,True,True,other,Moving Violation,Searched as a result of Probation or Parole Co...,Warning
885931,768652,2016-06-30,20:25:00,OFARRELL & JONES,37.785809,-122.412995,J,25.0,black,male,...,False,True,warning,True,True,True,other,Moving Violation,"Search without Consent, Positive Result",Warning
885988,768706,2016-06-30,23:40:00,FLORIDA ST & TREAT ST,37.768711,-122.411788,D,30.0,black,female,...,False,True,warning,False,True,True,other,DUI Check,"Search without Consent, Negative Result",Warning


In [109]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame)
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [111]:
clean_dataset(sf[['subject_age']])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,subject_age
33182,39.0
33185,24.0
33192,24.0
33210,27.0
33211,48.0
...,...
885924,59.0
885927,35.0
885931,25.0
885988,30.0


In [112]:
sf.outcome.value_counts(normalize=True)

citation    0.546634
warning     0.293926
arrest      0.159440
Name: outcome, dtype: float64

In [113]:
sf['subject_race'].astype('category')
sf['subject_sex'].astype('category')

33182       male
33185     female
33192       male
33210       male
33211       male
           ...  
885924      male
885927      male
885931      male
885988    female
886035      male
Name: subject_sex, Length: 41075, dtype: category
Categories (2, object): [female, male]

In [76]:
arrest = sf[sf.outcome == 'arrest']

In [77]:
arrest.subject_race.value_counts(normalize=True)

white                     0.332914
black                     0.296855
hispanic                  0.191530
asian/pacific islander    0.102390
other                     0.076310
Name: subject_race, dtype: float64

In [78]:
arrest.subject_age.describe()

count    11320.000000
mean        34.579682
std         12.357852
min         10.000000
25%         25.000000
50%         32.000000
75%         42.000000
max         99.000000
Name: subject_age, dtype: float64

In [79]:
youth_arrests = arrest[arrest.subject_age < 25]

In [80]:
youth.subject_race.value_counts(normalize=True)

black                     0.338609
white                     0.254693
hispanic                  0.224512
asian/pacific islander    0.104159
other                     0.078027
Name: subject_race, dtype: float64

In [12]:
youth_arrests.reason_for_stop.value_counts()

Moving Violation                                              1416
Mechanical or Non-Moving Violation (V.C.)                     1073
BOLO/APB/Warrant                                                68
Traffic Collision                                               24
DUI Check                                                       21
MPC Violation                                                   10
Assistance to Motorist                                           9
Moving Violation|Mechanical or Non-Moving Violation (V.C.)       2
NA|Moving Violation                                              1
Name: reason_for_stop, dtype: int64

In [81]:
youth_arrests.subject_sex.value_counts()

male      2118
female     599
Name: subject_sex, dtype: int64

In [82]:
sf.columns

Index(['raw_row_number', 'date', 'time', 'location', 'lat', 'lng', 'district',
       'subject_age', 'subject_race', 'subject_sex', 'type', 'arrest_made',
       'citation_issued', 'warning_issued', 'outcome', 'contraband_found',
       'search_conducted', 'search_vehicle', 'search_basis', 'reason_for_stop',
       'raw_search_vehicle_description', 'raw_result_of_contact_description',
       'arrested', 'young_and_black'],
      dtype='object')

In [83]:
sf.subject_sex.dtype

dtype('O')

In [84]:
sf['arrested'] = sf.outcome == 'arrest'
sf['young_and_black'] = youth.subject_race == 'black'

In [85]:
sf['subject_race'].astype('category')
sf['subject_sex'].astype('category')

0         female
1           male
2           male
3           male
4           male
           ...  
905065      male
905066      male
905067      male
905068      male
905069      male
Name: subject_sex, Length: 905070, dtype: category
Categories (2, object): [female, male]

In [66]:
X = sf[['young_and_black','subject_sex','subject_age']]
y = sf.arrested

In [67]:
sf.subject_sex.head(10)

0    female
1      male
2      male
3      male
4      male
5      male
6      male
7    female
8      male
9    female
Name: subject_sex, dtype: object

In [114]:
enc = OneHotEncoder(handle_unknown='ignore')

In [115]:
X_encoded = X

In [116]:
enc.fit(X_encoded)

ValueError: Input contains NaN

,raw_row_number,date,time,location,lat,lng,district,subject_age,subject_race,subject_sex,...,citation_issued,warning_issued,outcome,contraband_found,search_conducted,search_vehicle,search_basis,reason_for_stop,raw_search_vehicle_description,raw_result_of_contact_description
33182,50,2007-01-01,13:10:00,GENEVA/MISSION,37.716407,-122.441087,H,39.0,hispanic,male,...,False,False,arrest,False,True,True,other,Moving Violation,"Vehicle Inventory, Negative Result",In Custody Arrest
33185,53,2007-01-01,13:57:00,13TH ST / MISSION ST,37.769954,-122.419962,D,24.0,hispanic,female,...,False,False,arrest,True,True,True,other,Mechanical or Non-Moving Violation (V.C.),"Search Incident to Arrest, Positive Result",In Custody Arrest
33192,59,2007-01-01,14:45:00,GREAT HWY/FULTON,37.771329,-122.511123,G,24.0,white,male,...,False,True,warning,False,True,True,other,Mechanical or Non-Moving Violation (V.C.),Searched as a result of Probation or Parole Co...,Warning
33210,76,2007-01-01,19:40:00,LEAVENWORTH AND GOLDEN GATE,37.781878,-122.413880,J,27.0,white,male,...,False,True,warning,False,True,True,consent,Mechanical or Non-Moving Violation (V.C.),"Search with Consent, Negative Result",Warning
33211,77,2007-01-01,20:00:00,EVANS AT THIRD STREET,37.742817,-122.388026,C,48.0,black,male,...,True,False,citation,False,True,True,other,Mechanical or Non-Moving Violation (V.C.),"Vehicle Inventory, Negative Result",Citation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885924,768645,2016-06-30,20:00:00,442 GEARY,37.787168,-122.410733,A,59.0,hispanic,male,...,True,False,citation,True,True,True,other,Moving Violation,"Search without Consent, Positive Result",Citation
885927,768648,2016-06-30,20:05:00,HYDE AT GOLDENGATE,37.781668,-122.415524,J,35.0,asian/pacific islander,male,...,False,True,warning,False,True,True,other,Moving Violation,Searched as a result of Probation or Parole Co...,Warning
885931,768652,2016-06-30,20:25:00,OFARRELL & JONES,37.785809,-122.412995,J,25.0,black,male,...,False,True,warning,True,True,True,other,Moving Violation,"Search without Consent, Positive Result",Warning
885988,768706,2016-06-30,23:40:00,FLORIDA ST & TREAT ST,37.768711,-122.411788,D,30.0,black,female,...,False,True,warning,False,True,True,other,DUI Check,"Search without Consent, Negative Result",Warning
